In [11]:
import torch
from torchdiffeq import odeint_adjoint as odeint
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import sys
sys.path.append('../')
from typing import Tuple, Any

%load_ext autoreload
%autoreload 2


import interflow as itf
import interflow.prior as prior
import interflow.fabrics
import interflow.stochastic_interpolant as stochastic_interpolant
import interflow.coupling_flows as coupling_flows
from interflow.coupling_flows import *
from torch import autograd
from functorch import jacfwd, vmap


if torch.cuda.is_available():
    print('CUDA available, setting default tensor residence to GPU.')
    itf.util.set_torch_device('cuda')
else:
    print('No CUDA device found!')
print(itf.util.get_torch_device())


print("Torch version:", torch.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA available, setting default tensor residence to GPU.
cuda
Torch version: 2.5.1


In [12]:
C,H,W = 3,32,32
flow = TimeIndexedAffineFlow(shape=(C,H,W), num_layers=16,
                             img_hidden_channels=96, img_blocks=3).cuda()
x = torch.randn(8, C, H, W, device="cuda")
t = torch.rand(8, device="cuda")
y, logdet = flow(x, t)                 # forward, preserves [B,C,H,W]
dydt = flow.dTdt(x, t, method="autograd")
trJ  = flow.trace_jacobian_x(x, t, num_samples=4)


RuntimeError: vmap: index_put_(self, *extra_args) is not possible because there exists a Tensor `other` in extra_args that has more elements than `self`. This happened due to `other` being vmapped over but `self` not being vmapped over in a vmap. Please try to use out-of-place operators instead of index_put_. If said operator is being called inside the PyTorch framework, please file a bug report instead.

In [8]:
y

tensor([[-0.0409,  0.3887,  0.9579,  ...,  0.6504, -0.1627, -0.9828],
        [-0.9619, -0.0501, -0.6224,  ..., -0.1594, -0.0154, -1.0066],
        [ 0.7854,  0.9128, -1.2187,  ..., -0.3365, -0.9046, -1.0761],
        ...,
        [-0.0382, -0.7440,  0.5124,  ..., -0.4663,  0.4113, -0.5843],
        [ 0.3533,  0.2511, -0.3202,  ...,  1.2813, -0.3715,  1.1674],
        [-0.6010, -0.5538, -1.0257,  ..., -0.1993,  0.5952,  1.9400]],
       grad_fn=<IndexSelectBackward0>)